**Perform Partial dependence plot (PDP)**

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import PartialDependenceDisplay
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv('cali.csv')
df.head()

In [ ]:
X=df.drop(columns='target')
y=df['target']

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

model=RandomForestRegressor(n_estimators=100,random_state=42)
model.fit(X_train,y_train)

In [ ]:
PartialDependenceDisplay.from_estimator(model,X_train,features=['MedInc'],feature_names=df.columns)
plt.show()

In [ ]:
PartialDependenceDisplay.from_estimator(model,X_train,[('MedInc','HouseAge')],feature_names=df.columns)
plt.show()

**LIME FOR IMAGE**

In [ ]:
from tensorflow.keras.applications.xception import Xception,decode_predictions
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense
from tensorflow.keras.utils import to_categorical
from lime import lime_image
import seaborn as sns
from skimage.color import label2rgb
import warnings
warnings.filterwarnings('ignore')

In [ ]:
(x_train,y_train),(x_test,y_test)=mnist.load_data()

x_train=x_train.astype('float32')/255.0
x_test=x_test.astype('float32')/255.0

y_train_cat=to_categorical(y_train,10)
y_test_cat=to_categorical(y_test,10)

In [ ]:
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train_cat, epochs=2, batch_size=64, validation_split=0.1)


In [ ]:
from lime import lime_image
from skimage.color import label2rgb
import cv2
import numpy as np
import matplotlib.pyplot as plt
explainer = lime_image.LimeImageExplainer()
idx = 25
test_image = x_test[idx]

# Convert LIME's RGB input back to grayscale
predict_fn = lambda x: model.predict(
    np.array([cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) for img in x])[..., np.newaxis]
)

explanation = explainer.explain_instance(
    image=test_image.squeeze(),  # shape: (28, 28)
    classifier_fn=predict_fn,
    top_labels=1,
    hide_color=0,
    num_samples=1000
)

temp, mask = explanation.get_image_and_mask(
    label=explanation.top_labels[0],
    positive_only=True,
    hide_rest=False
)

plt.imshow(label2rgb(mask, temp, bg_label=0))
plt.title(f"LIME for label: {y_test[idx]}")
plt.axis('off')
plt.show()
